In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Assignment 3: Improving the Pipeline

## Things to fix in my pipeline
1. ~~A way to read different types of data.~~
2. ~~Make it so that the plotting functions have a title.~~
3. A pre-processing function that handles outliers.
4. ~~A function that discretizes continuous features,~~ and ~~one that turns a categorical variable into dummies.~~
5. ~~Add more classifiers to the pipeline~~
    * Added logit, KNN, decision trees, SVM, random forests, naive bayes, and gradient boostered trees.
6. ~~Experiment with different parameters for these classifiers.~~
7. ~~Add additional evaluation metrics.~~
8. ~~Create temporal validation function.~~ (sort of)

## Analysis

First, we need to pre-process the data set. We begin for filtering the data for the years we're interested in, 2011-2013. Then, we create the outcome labels we're interested in (whether or not a project got fully funded).

In [2]:
import pipeline_v2 as pxy

In [5]:
df = pxy.read_data("projects.csv", "csv")
ldf = pxy.read_data("outcomes.csv", "csv")
df['year_dt'] = pxy.create_datetime(df, 'date_posted')
df['year_posted'] = pxy.retrieve_year(df, 'year_dt')

In [6]:
# filter for data from 2011-2013
df2 = df.loc[(df['year_posted']>2010) & (df['year_posted']<2014),:]

In [7]:
def left_merge(df_left, df_right, merge_column):
    return pd.merge(df_left, df_right, how = 'left', on = merge_column)

In [10]:
maindata = left_merge(df2, ldf, "projectid")

In [11]:
maindata['true_labs'] = np.nan
for i in range(0, len(maindata)):
    if maindata.loc[i,'fully_funded'] == "t":
        maindata.loc[i,'true_labs'] = 1
    if maindata.loc[i,'fully_funded'] == "f":
        maindata.loc[i,'true_labs'] = 0

KeyboardInterrupt: 

In [ ]:
maindata['month_posted'] =pxy.retrieve_month(df, 'year_dt')

In [ ]:
subdata = pxy.take_sample(maindata, 0.02).reset_index()

## Splitting the data into training and test sets:

First I split my data into the training set and test set. The training set consists of projects that were posted from 1/2011-6/2013, while the test set comprises projects from 7/2013 to 12/2013.

## Temporal cross-validation over the training set

I will also be performing temporal cross-validation over the training set, using the TimeSeriesSplit function from sklearn.

In [3]:
maindata = pxy.read_data("main_data.csv", "csv")

In [4]:
from datetime import datetime

In [5]:
cutoff_dt = datetime(2013, 7, 1, 0, 0)

In [6]:
prelim_featlist = ['teacher_teach_for_america', # dum
                   'school_charter', # dum
                   'school_magnet', # dum
                   'primary_focus_subject', # cat
                    'resource_type', # cat
                   'poverty_level', # cat
                   'grade_level', # cat
                   'total_price_excluding_optional_support', # cont
                   'students_reached' # cont
                  ]

## Feature generation

In [7]:
df_feat1 = maindata[prelim_featlist]

In [8]:
df_feat2 = pxy.generate_dummy(
    pxy.generate_dummy(pxy.generate_dummy(df_feat1, 'primary_focus_subject', attach= True),
    'poverty_level', attach = True),
    'grade_level', attach = True
)

In [9]:
df_feat3 = df_feat2.drop(columns = ['primary_focus_subject', 'poverty_level', 'grade_level'])

In [10]:
pd.set_option("display.max_columns",999)

In [11]:
# recode dummies to binary
for i in range(0,3):
    feat_name = prelim_featlist[i]
    for row_i in range(0, len(df_feat3)):
        if df_feat3.loc[row_i,prelim_featlist[i]] == "t":
            df_feat3.loc[row_i,prelim_featlist[i]] = 1
        else:
            df_feat3.loc[row_i,prelim_featlist[i]] = 0


KeyboardInterrupt



In [ ]:
df_features = pxy.generate_dummy(df_feat3, "resource_type", attach = True).drop(columns = ['resource_type'])

In [ ]:
df_features.to_csv("df_features_done.csv")

In [ ]:
#### reset and reload df_features

In [12]:
df_feats = pd.read_csv("df_features_done.csv")
df_feats = df_feats.set_index("Unnamed: 0")

In [13]:
# i did write a function based on this afterwards
for i in range(0,46):
    df_feats.iloc[:,i] = df_feats.iloc[:,i].fillna(df_feats.iloc[:,i].mode()[0])

In [19]:
df_feats_final = pd.concat([df_feats, maindata['year_dt']], axis =1)

Form the main training and testing set:

In [20]:
df_outcomes = maindata[['true_labs', 'year_dt']]

In [21]:
df_feats_final['year_dt'] = pxy.create_datetime(df_feats_final ,'year_dt')
df_outcomes['year_dt'] = pxy.create_datetime(df_outcomes ,'year_dt')

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
# don't drop dt column for training sets b/c we still need them ...
x_train = df_feats_final.loc[df_feats_final['year_dt'] < cutoff_dt,:]
x_test = df_feats_final.loc[df_feats_final['year_dt'] >= cutoff_dt,:].drop(columns = 'year_dt')

In [23]:
y_train = df_outcomes.loc[df_outcomes['year_dt'] < cutoff_dt,:]
y_test = df_outcomes.loc[df_outcomes['year_dt'] >= cutoff_dt,:].drop(columns = 'year_dt')

Create temporal cross validation

In [24]:
cv_time_thresholds = [2012, 2013]
cv_dict = {"iter_1":{}, "iter_2":{}}

In [25]:
xcv1_train = x_train.loc[x_train['year_dt'] < datetime(2012,1,1),:].drop(columns = 'year_dt')
xcv1_test = x_train.loc[(x_train['year_dt'] < datetime(2012,1,1)) & (x_train['year_dt'] < datetime(2013,1,1)),:].drop(columns = 'year_dt')
xcv2_train = x_train.loc[x_train['year_dt'] < datetime(2013,1,1),:].drop(columns = 'year_dt')
xcv2_test = x_train.loc[x_train['year_dt'] >= datetime(2013,1,1),:].drop(columns = 'year_dt')

In [26]:
ycv1_train = y_train.loc[y_train['year_dt'] < datetime(2012,1,1),:].drop(columns = 'year_dt')
ycv1_test = y_train.loc[(y_train['year_dt'] < datetime(2012,1,1)) & (y_train['year_dt'] < datetime(2013,1,1)),:].drop(columns = 'year_dt')
ycv2_train = y_train.loc[y_train['year_dt'] < datetime(2013,1,1),:].drop(columns = 'year_dt')
ycv2_test = y_train.loc[y_train['year_dt'] >= datetime(2013,1,1),:].drop(columns = 'year_dt')

In [27]:
model_params = {
    'max_features': ["auto"], 
    'n_estimators' : [5, 10, 20, 50, 100], 
    "max_depth": [3,5,8], 
    "criterion": ["gini", "entropy"]
    }

In [28]:
def custom_predictions(clf, threshold, x_test):
    # threshold = the probability threshold for something to be a 0.
    # generate array with predicted probabilities
    pred_array = clf.predict_proba(x_test)

    # initialize an empty array for the predictions
    pred_generated = np.array([])

    # predict the first entry
    if pred_array[0][0] >= threshold:
        pred_generated = np.hstack([pred_generated, 0])
    else:
        pred_generated = np.hstack([pred_generated, 1])

    # loops over the rest of the array
    for i in range(1,len(x_test)):
        if pred_array[i][0] >= threshold:
            pred_generated = np.vstack([pred_generated, 0])
        else:
            pred_generated = np.vstack([pred_generated, 1])

    # return an np.array
    return  pred_generated

In [29]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score

In [64]:
def run_cv_rf(threshold_list, model_params):
    results_rf = {}
    max_feats, num_dtrees_list, depth_list, criterion_list = model_params['max_features'], model_params['n_estimators'],  model_params["max_depth"], model_params["criterion"] 
    for threshold_i in threshold_list:
         for feature_number in max_feats:
            # loop over n_estimators
            for num_trees in num_dtrees_list:
                #loop over max_depth
                for depth_num in depth_list:
                    #loop over information gain criterion
                    for criterion_choice in criterion_list:
                        rf_clf = RandomForestClassifier(max_features = feature_number, n_estimators = num_trees, max_depth = depth_num, criterion = criterion_choice)
                        rf_clf.fit(xcv1_train, ycv1_train)
                        y_pred = custom_predictions(rf_clf, threshold = threshold_i, x_test = xcv1_test)
                        prec1 = precision_score(ycv1_test, y_pred)
                        rec1 = recall_score(ycv1_test, y_pred)
                        acc1 = accuracy_score(ycv1_test, y_pred)
                        try:
                            roc1 = roc_auc_score(ycv1_test, y_pred)
                        except ValueError:
                            roc1 = 9999999
                        rf_clf = RandomForestClassifier(max_features = feature_number, n_estimators = num_trees, max_depth = depth_num, criterion = criterion_choice)
                        rf_clf.fit(xcv2_train, ycv2_train)
                        y_pred = custom_predictions(rf_clf, threshold = threshold_i, x_test = xcv2_test)
                        # intialize the main dictionary key
                        model_key = ("Random Forest", feature_number, num_trees, depth_num, criterion_choice, threshold_i)
                        prec2 = precision_score(ycv2_test, y_pred)
                        rec2 = recall_score(ycv2_test, y_pred)
                        acc2 = accuracy_score(ycv2_test, y_pred)
                        try:
                            roc2 = roc_auc_score(ycv2_test, y_pred)
                        except ValueError:
                            roc2 = 9999999
                        
                        results_rf[model_key] = {}
                        # write evaluation results to dictionary
                        results_rf[model_key]['Precision']= (prec1 + prec2)/2
                        results_rf[model_key]['Recall']= (rec1 + rec2)/2
                        results_rf[model_key]['AUC Score'] = (roc1 + roc2)/2
                        results_rf[model_key]['Accuracy'] = (acc1 + acc2)/2
    return results_rf

In [65]:
thresh_list = [0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.5]

In [66]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

In [67]:
results_rf = run_cv_rf(thresh_list, model_params)

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3

...

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/si

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib

In [68]:
rf_df = pd.DataFrame(results_rf)

In [69]:
rf_df.to_csv("rf_results.csv")

In [70]:
def run_cv_lr(threshold_list):
    results_lr = {}
    penalty_list = ['l1', 'l2']
    cvalues_list = [10**-5, 10**-2, 10**-1, 1, 10, 10**2, 10**5]
    max_feats, num_dtrees_list, depth_list, criterion_list = model_params['max_features'], model_params['n_estimators'],  model_params["max_depth"], model_params["criterion"] 
    for threshold_i in threshold_list:
        for c_para in cvalues_list:
            for penalty_para in penalty_list:
                #initialize logistic regression object with combination of parameters
                logreg = LogisticRegression(penalty = penalty_para, C = c_para)
                logreg.fit(xcv1_train, ycv1_train)
                y_pred = custom_predictions(logreg, threshold = threshold_i, x_test = xcv1_test)                

                prec1 = precision_score(ycv1_test, y_pred)
                rec1 = recall_score(ycv1_test, y_pred)
                acc1 = accuracy_score(ycv1_test, y_pred)
                try:
                    roc1 = roc_auc_score(ycv1_test, y_pred)
                except ValueError:
                    roc1 = 9999999                

                logreg = LogisticRegression(penalty = penalty_para, C = c_para)
                logreg.fit(xcv2_train, ycv2_train)
                y_pred = custom_predictions(logreg, threshold = threshold_i, x_test = xcv2_test)                

                prec2 = precision_score(ycv2_test, y_pred)
                rec2 = recall_score(ycv2_test, y_pred)
                acc2 = accuracy_score(ycv2_test, y_pred)
                try:
                    roc2 = roc_auc_score(ycv2_test, y_pred)
                except ValueError:
                    roc2 = 9999999      

                # intialize the main dictionary key
                model_key = ("LR", c_para, penalty_para, threshold_i)
                
                results_lr[model_key] = {}
                # write evaluation results to dictionary
                results_lr[model_key]['Precision']= (prec1 + prec2)/2
                results_lr[model_key]['Recall']= (rec1 + rec2)/2
                results_lr[model_key]['AUC Score'] = (roc1 + roc2)/2
                results_lr[model_key]['Accuracy'] = (acc1 + acc2)/2

    return results_lr

In [71]:
results_lr1 = run_cv_lr(thresh_list)

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [72]:
lr_df = pd.DataFrame(results_lr1)

In [73]:
lr_df.to_csv("lr_results.csv")

In [51]:
## the baseline
y_baseline_pred = y_train.copy()

In [52]:
y_baseline_pred['base_pred'] = 1

In [53]:
y_baseline = y_baseline_pred['base_pred']

In [62]:
?roc_auc_score

In [59]:
precision_score(y_train['true_labs'], y_baseline)

0.6975627978582506

In [60]:
recall_score(y_train['true_labs'], y_baseline)

1.0

# Report

Brief reflection: it took me quite some time to figure out how to do the temporal validation. I tried gridsearchcv and timeseriesplit to ultimately no avail. I hacked together my own cross validation function, but I'll have to think harder about constructing a generalizable function for my pipeline.

## Who are the best performers?

Let's assume that the baseline predictor guesses that every project gets fully funded. Such a predictor would have a precision of 0.697 and recall of 0.5, when we intervene on 50% of the population.

In [ ]:
rf_df = pd.read_csv("rf_results.csv")
lr_df = pd.read_csv("lr_results.csv")

In [85]:
rf_df.transpose().sort_values("Accuracy", ascending = False).head(10)

AUC Score  Accuracy  Precision    Recall
Random Forest auto 5   8 gini    0.5   0.504823  0.683554   0.683879  0.996401
                   10  8 gini    0.5   0.503514  0.683525   0.683293  0.998659
                   5   8 entropy 0.5   0.502442  0.682971   0.682818  0.999044
                   10  8 entropy 0.5   0.501994  0.682868   0.682622  0.999548
                   20  8 entropy 0.5   0.501581  0.682666   0.682441  0.999724
                   100 8 gini    0.5   0.500960  0.682321   0.682169  0.999873
                         entropy 0.5   0.500840  0.682254   0.682116  0.999901
                   50  8 entropy 0.5   0.500669  0.682129   0.682042  0.999859
                         gini    0.5   0.500529  0.682062   0.681981  0.999922
                   20  8 gini    0.5   0.500515  0.682043   0.681975  0.999894

In [84]:
lr_df.transpose().sort_values("Accuracy", ascending = False).head(10)

AUC Score  Accuracy  Precision    Recall
LR 100.0    l2 0.5   0.519267  0.686435   0.690457  0.978923
   1.0      l1 0.5   0.519477  0.686434   0.690550  0.978568
   100.0    l1 0.5   0.519554  0.686426   0.690588  0.978410
   100000.0 l1 0.5   0.519548  0.686424   0.690585  0.978414
   10.0     l1 0.5   0.519525  0.686424   0.690574  0.978455
   1.0      l2 0.5   0.519447  0.686363   0.690539  0.978424
   0.1      l1 0.5   0.518730  0.686245   0.690197  0.979370
   100000.0 l2 0.5   0.518283  0.686051   0.689991  0.979618
   10.0     l2 0.5   0.518434  0.686032   0.690060  0.979311
   0.1      l2 0.5   0.518090  0.685973   0.689901  0.979747

The best performing classifier in terms to accuracy are LR classifier (C = 1.0, L1 regularization). It seems like it performs pretty decently: high recall, and 0.69 in terms of precision. Of course, accuracy assumes that we care about false positives and false negatives equally: as we can see, they have AUC-ROC scores near 0.5. That's a big assumption.

In [87]:
rf_df.transpose().sort_values("Precision", ascending = False).head()

AUC Score  Accuracy  Precision    Recall
Random Forest auto 10 8 gini    0.10   0.500099  0.318385   1.000000  0.000198
                   5  8 gini    0.05   0.500012  0.318267   1.000000  0.000024
                   20 8 entropy 0.10   0.500373  0.318762   0.994253  0.000760
                   10 8 entropy 0.10   0.501241  0.319961   0.991803  0.002542
                   20 3 entropy 0.20   0.500621  0.319142   0.977387  0.001377

In [86]:
lr_df.transpose().sort_values("Precision", ascending = False).head()

AUC Score  Accuracy  Precision    Recall
LR 0.01  l2 0.10   0.500918  0.319547   0.985149  0.001947
   1.00  l2 0.05   0.500549  0.319007   0.981481  0.001114
   0.01  l1 0.10   0.501047  0.319805   0.969799  0.002424
   0.10  l2 0.10   0.501851  0.321007   0.941168  0.004329
   10.00 l2 0.10   0.501531  0.320564   0.935817  0.003665

Comparing the top performers in terms of precision, we can see that that random forests outperform LR slightly. But we can clearly see a problem with looking exclusively at precision: it's easy to get high precision, but it comes at the cost of recall — in addition, we can only achieve that when we choose to act on a lower threshold of the population (all of the high performers do so on pop % < 0.2, for example). So let's not do that. We clearly don't care about precision *that* much in this case!

In [88]:
rf_df.transpose().sort_values("Recall", ascending = False).head(5)

AUC Score  Accuracy  Precision  Recall
Random Forest auto 20  3 gini    0.5        0.5   0.68175    0.68175     1.0
                   5   3 gini    0.5        0.5   0.68175    0.68175     1.0
                   10  3 entropy 0.5        0.5   0.68175    0.68175     1.0
                         gini    0.5        0.5   0.68175    0.68175     1.0
                   100 3 gini    0.5        0.5   0.68175    0.68175     1.0

In [89]:
lr_df.transpose().sort_values("Recall", ascending = False).head(5)

AUC Score  Accuracy  Precision    Recall
LR 0.00001 l2 0.5   0.500000  0.681750   0.681750  1.000000
           l1 0.5   0.500000  0.681750   0.681750  1.000000
   0.01000 l1 0.5   0.514977  0.685290   0.688446  0.983383
           l2 0.5   0.517219  0.685783   0.689497  0.980755
   0.10000 l2 0.5   0.518090  0.685973   0.689901  0.979747

Similar issue as above: it seems trivial to maximize recall, but it comes at the cost of other metrics we might want to care about. Again, we don't care about recall *that* much.

In [81]:
rf_df.transpose().sort_values("AUC Score", ascending = False).head(10)

AUC Score  Accuracy  Precision    Recall
Random Forest auto 100 8 gini    0.3   0.623707  0.582204   0.806609  0.509315
                         entropy 0.3   0.622625  0.577277   0.809052  0.497491
                   50  8 entropy 0.3   0.622176  0.575913   0.809335  0.494534
                   10  8 entropy 0.3   0.621046  0.572375   0.810272  0.486862
                   20  8 gini    0.3   0.620329  0.572457   0.809068  0.488216
                   50  8 gini    0.3   0.620180  0.574734   0.806913  0.494742
                   10  8 gini    0.3   0.619625  0.584010   0.799589  0.520994
                   20  8 entropy 0.3   0.616108  0.573881   0.801483  0.499322
                       5 gini    0.3   0.615468  0.572242   0.800176  0.496471
                   100 5 entropy 0.3   0.614386  0.565305   0.804276  0.478864

In [80]:
lr_df.transpose().sort_values("AUC Score", ascending = False).head(10)

AUC Score  Accuracy  Precision    Recall
LR 0.01      l1 0.3   0.608410  0.591560   0.778014  0.561464
   0.10      l1 0.3   0.607926  0.592754   0.776577  0.565688
   100.00    l2 0.3   0.607854  0.593707   0.775932  0.568460
   100000.00 l1 0.3   0.607728  0.593248   0.775996  0.567410
   100.00    l1 0.3   0.607725  0.593268   0.775981  0.567470
   10.00     l1 0.3   0.607695  0.593200   0.775974  0.567336
   1.00      l2 0.3   0.607605  0.592950   0.776038  0.566787
             l1 0.3   0.607547  0.592939   0.775921  0.566876
   0.10      l2 0.3   0.607375  0.592828   0.775785  0.566865
   10.00     l2 0.3   0.607128  0.592920   0.775491  0.567520

My final thought is that I don't quite know how to pick the best performers! I guess I'd assume that this would be a trivial matter. But this is absolutely not the case. This situation will only be compounded when I have more classifiers to choose from, and matters of interpretability come into consideration as well.

# Settling on a choice:

In reality, I guess I have to settle on an ad-hoc set of criteria. Let's say I want to be able to predict at least 80% of the true positive cases (recall >0.8). And within that, I want to be as precise as possible. Given this criteria, what would I pick?

In [91]:
lrdf = lr_df.transpose()
rfdf = rf_df.transpose()

In [93]:
lrdf.loc[lrdf['Recall']>0.8,:].sort_values("Precision", ascending = False).head()

AUC Score  Accuracy  Precision    Recall
LR 100.0    l1 0.5   0.519554  0.686426   0.690588  0.978410
   100000.0 l1 0.5   0.519548  0.686424   0.690585  0.978414
   10.0     l1 0.5   0.519525  0.686424   0.690574  0.978455
   1.0      l1 0.5   0.519477  0.686434   0.690550  0.978568
            l2 0.5   0.519447  0.686363   0.690539  0.978424

In [94]:
rfdf.loc[rfdf['Recall']>0.8,:].sort_values("Precision", ascending = False).head()

AUC Score  Accuracy  Precision    Recall
Random Forest auto 5  8 gini    0.5   0.504823  0.683554   0.683879  0.996401
                   10 8 gini    0.5   0.503514  0.683525   0.683293  0.998659
                   5  8 entropy 0.5   0.502442  0.682971   0.682818  0.999044
                   10 8 entropy 0.5   0.501994  0.682868   0.682622  0.999548
                   20 8 entropy 0.5   0.501581  0.682666   0.682441  0.999724

Given that, it would seem that a LR classifier with C = 100 and L1 regularization, acting at a threshold of 0.5 (50% of the population) would work best.